In [1]:
# Cell 1: Import Libraries and Authenticate
# -------------------------------------------
# We start by importing the necessary libraries. `asf_search` is for searching the
# ASF archive, `os` is for interacting with the file system, and `getpass`
# will allow us to securely enter our Earthdata Login credentials.

import asf_search as asf
import os
from getpass import getpass

# It's recommended to authenticate with your Earthdata Login credentials.
# This will prevent you from being rate-limited when downloading data.
# The `getpass` function will prompt you to securely enter your username and password.
try:
    # Attempt to get credentials from environment variables first
    username = os.environ['EARTHDATA_USERNAME']
    password = os.environ['EARTHDATA_PASSWORD']
    print("Using Earthdata credentials from environment variables.")
except KeyError:
    # If not found, prompt the user
    print("Please enter your Earthdata Login credentials.")
    username = getpass('Username: ')
    password = getpass('Password: ')

# Create a session with your credentials
try:
    session = asf.ASFSession().auth_with_creds(username, password)
    print("Successfully authenticated with Earthdata Login.")
except Exception as e:
    print(f"Authentication failed: {e}")
    # You can still search without authentication, but downloads might be limited.
    session = asf.ASFSession()

Please enter your Earthdata Login credentials.


Username:  ········
Password:  ········


Successfully authenticated with Earthdata Login.


In [2]:
# Cell 2: Define Search Parameters
# --------------------------------
# Here, we define our Area of Interest (AOI) as a bounding box and the date range
# for our scene search.

# Bounding Box [min_lon, min_lat, max_lon, max_lat]
bbox = [22.92550833727161, 38.7509441550928, 23.563700414775237, 39.08766821216534]

# Convert bbox to a WKT string for the asf_search library
wkt_aoi = f"POLYGON (({bbox[0]} {bbox[1]}, {bbox[2]} {bbox[1]}, {bbox[2]} {bbox[3]}, {bbox[0]} {bbox[3]}, {bbox[0]} {bbox[1]}))"


# Date range for the search
start_date = "2021-07-29"
end_date = "2021-08-22"

# Create a directory to store our data if it doesn't exist
if not os.path.exists("s1_data"):
    os.makedirs("s1_data")
    print("Created directory: 's1_data'")

In [3]:
# Cell 3: Perform the Search
# --------------------------
# Now we use our defined parameters to search for Sentinel-1 GRD products.

print("\nSearching for Sentinel-1 scenes...")
results = asf.geo_search(
    platform=[asf.PLATFORM.SENTINEL1],
    intersectsWith=wkt_aoi,
    start=start_date,
    end=end_date,
    processingLevel=[asf.PRODUCT_TYPE.GRD_HD], # Example: Ground Range Detected, High-Resolution
    beamMode=[asf.BEAMMODE.IW], # Example: Interferometric Wide swath
    flightDirection=asf.FLIGHT_DIRECTION.DESCENDING,
)

print(f"Found {len(results)} scenes.")

# You can inspect the results if you want
# This loop will print the name of each scene found.
print("\nInspecting found scenes:")
for product in results:
    print(f"  - {product.properties['sceneName']}")


Searching for Sentinel-1 scenes...
Found 6 scenes.

Inspecting found scenes:
  - S1A_IW_GRDH_1SDV_20210818T043145_20210818T043210_039279_04A35A_6BBD
  - S1B_IW_GRDH_1SDV_20210812T043117_20210812T043142_028208_035D7A_9D15
  - S1B_IW_GRDH_1SDV_20210812T043052_20210812T043117_028208_035D7A_71FB
  - S1A_IW_GRDH_1SDV_20210806T043144_20210806T043209_039104_049D48_3027
  - S1B_IW_GRDH_1SDV_20210731T043116_20210731T043141_028033_035815_3C8B
  - S1B_IW_GRDH_1SDV_20210731T043051_20210731T043116_028033_035815_29C7


In [5]:
# Cell 4: Download the Data
# -------------------------
# Finally, we can download the scenes found in our search.

if results:
    print("\nStarting download...")
    # The `download` method takes the search results, the target directory,
    # and the session object with our authentication.
    results.download(path="s1_data", session=session)
    print("Download complete.")
else:
    print("\nNo scenes found to download for the given parameters.")


Starting download...
Download complete.
